### 2022 Fintech Summer Research
### Step 2: Basic GUIs
### David Park
### Last Updated: Thursday, July 14, 2022

Imports


In [1]:
import PySimpleGUI as sg
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.pyplot import figure
import pprint
import os
import pandas as pd
import seaborn as sns
import numpy as np


### Stock Data Retrieval Functions
Basic Graph Data
###### Takes in stock name, HFT trading interval, and date interval up to 1 week, the start day (str), and the end day (str)
###### Returns a list of the DataFrame, start day (str), end day (str), and stock name (str)
###### EX: graph_data('AAPL', '1m', '2022-07-10', '2022-07-14')

In [2]:
def graph_data(name, tf, start_day, end_day):
    tickerData = yf.Ticker(name)
    # Retrieve Basic Data
    stock_df = tickerData.history(tickers = name, interval = tf, start=start_day, end=end_day)
    # Create Typical Price Column
    stock_df['Typical'] = (stock_df['High'] + stock_df['Low'] + stock_df['Close'])/3
    # Create closeOffHigh Column
    stock_df['closeOffHigh'] = 2 * ((stock_df['High'] - stock_df['Close'])/(stock_df['High'] - stock_df['Low'])) - 1
    # Create Volatility Column
    stock_df['Volatility'] = (stock_df['High'] - stock_df['Low'])/stock_df['Open']
    # Create LogReturn Column
    stock_df['LogReturn'] = np.log(stock_df['Close']/stock_df['Close'].shift(1))
    stock_df = stock_df.drop(['Dividends', 'Stock Splits'], axis=1)
    return [stock_df, start_day, end_day, name]

Example DataFrame & Plot

In [3]:
df = graph_data('AAPL', '1m', '2022-07-10', '2022-07-14')[0]
df
# plt.plot(df['Close'])
# plt.title('AAPL Close')


,Open,High,Low,Close,Volume,Typical,closeOffHigh,Volatility,LogReturn
Datetime,,,,,,,,,
2022-07-11 09:30:00-04:00,145.669998,145.850006,145.130005,145.279999,1685904,145.420003,0.583351,0.004943,NaN
2022-07-11 09:31:00-04:00,145.279999,145.850006,145.279999,145.630096,430116,145.586700,-0.228397,0.003924,0.002407
2022-07-11 09:32:00-04:00,145.630005,145.850006,145.399994,145.729507,319478,145.659836,-0.464465,0.003090,0.000682
2022-07-11 09:33:00-04:00,145.699997,145.809998,145.580994,145.660004,223596,145.683665,0.309968,0.001572,-0.000477
2022-07-11 09:34:00-04:00,145.649994,145.764999,145.413193,145.419998,237053,145.532730,0.961312,0.002415,-0.001649
...,...,...,...,...,...,...,...,...,...
2022-07-13 15:56:00-04:00,145.779999,145.940002,145.729996,145.860001,288249,145.843333,-0.238102,0.001441,0.000583
2022-07-13 15:57:00-04:00,145.860001,145.960007,145.660004,145.809998,367205,145.810003,0.000051,0.002057,-0.000343
2022-07-13 15:58:00-04:00,145.800003,145.899994,145.660004,145.660004,512344,145.740000,1.000000,0.001646,-0.001029


### GUI Functions
Creating Line Plots
###### Takes in list of DataFrame + start day (str) + end day (str) + stock name (str), and desired column name (str)
###### Returns a line plot of said data as output 
###### EX: create_line_plot(stock_df, 'Close')

In [4]:
def create_line_plot(df_list, col):
    # Plots the column of the DataFrame vs DateTime
    df_list[0][col].plot(color = 'blue', kind = 'line')
    # Labels columns and relevant axes
    plt.rcParams["figure.figsize"] = (15,10)
    plt.title(col + ' vs. Datetime')
    plt.ylabel(df_list[3], fontsize = 14)
    plt.grid(True)    
    plt.xlim([df_list[1], df_list[2]])
    plt.tight_layout()

    # Save plots
    fig_name = 'LINE_' + col + "_" + df_list[3] + df_list[1] + df_list[2] + '.png'
    
    plt.savefig(fig_name)

    if os.path.isfile(fig_name):
        print('\n'+ fig_name +'. is saved on'+ os.getcwd())
    else:
        raise Exception('can not save file\n')

    # Returns the plot as an object
    return plt.gcf()

Example Line Plot

In [5]:
# create_line_plot(graph_data('GOOG', '1m','2022-06-20', '2022-06-27'), 'Typical')

Creating Box Plots
###### Takes in list of DataFrame + start day (str) + end day (str) + stock name (str), and desired column name (str)
###### Returns a box plot of said data as output 
###### EX: create_box_plot(stock_df, 'Close')

In [6]:
def create_box_plot(df_list, col):
    # Plots the column of the boxplot
    plt.boxplot(df_list[0][col])
    # Labels columns and relevant axes
    plt.rcParams["figure.figsize"] = (15,10)
    plt.title('Boxplot')
    plt.ylabel(df_list[3], fontsize = 14)
    plt.xlabel(col)
    plt.grid(True)    
    plt.tight_layout()

    # Save plots
    fig_name = 'BOX_' + col + "_" + df_list[3] + df_list[1] + df_list[2] + '.png'
    
    plt.savefig(fig_name)

    if os.path.isfile(fig_name):
        print('\n'+ fig_name +'. is saved on'+ os.getcwd())
    else:
        raise Exception('can not save file\n')

    # Returns the plot as an object
    return plt.gcf()

Example Boxplot

In [7]:
# create_box_plot(graph_data('AAPL', '1m','2022-06-20', '2022-06-27'), 'LogReturn')


Creating Probability Density Functions
###### Takes in list of DataFrame + start day (str) + end day (str) + stock name (str), and desired column name (str)
###### Returns a PDF  of said data as output 
###### EX: create_pdf(stock_df, 'Close')

In [8]:
def create_pdf(df_list, col):
    # Plots the column of the boxplot
    sns.kdeplot(df_list[0][col], color = 'blue')
    # Labels columns and relevant axes
    plt.rcParams["figure.figsize"] = (15,10)
    plt.title('PDF')
    plt.ylabel(df_list[3], fontsize = 14)
    plt.xlabel(col)
    plt.grid(True)    
    plt.tight_layout()

    # Save plots
    fig_name = 'PDF_' + col + "_" + df_list[3] + df_list[1] + df_list[2] + '.png'
    
    plt.savefig(fig_name)

    if os.path.isfile(fig_name):
        print('\n'+ fig_name +'. is saved on'+ os.getcwd())
    else:
        raise Exception('can not save file\n')

    # Returns the plot as an object
    return plt.gcf()

In [9]:
# create_pdf(graph_data('AAPL', '1m','2022-06-20', '2022-06-27'), 'Typical')

Bollinger Bands
###### Takes in stock name, HFT trading interval, and date interval up to 1 week
###### Returns a list of the DataFrame, start day (str), end day (str), and stock name (str)
###### EX: bollinger_bands('AAPL', '1m', '2022-07-10', '2022-07-14')

In [10]:
def create_bollinger_bands(df_list, col):
    # Retrieve Basic Data
    stock_df = df_list[0]
    # Create Typical Price Column
    tp_data = stock_df[col].to_frame()

    # Create 20-minute Simple Moving Average Column and Moving Stddev Column
    # https://www.geeksforgeeks.org/how-to-calculate-moving-average-in-a-pandas-dataframe/
    
    tp_data['Moving AVG'] = tp_data[col].rolling(20).mean()
    tp_data['STDDEV20'] = tp_data[col].rolling(20).std()

    # Create Columns for Upper and Lower Bollinger Bands
    tp_data['Upper'] = tp_data['Moving AVG'] + 2 * tp_data['STDDEV20']
    tp_data['Lower'] = tp_data['Moving AVG'] - 2 * tp_data['STDDEV20']

    # Drop Typical Price Column
    tp_data = tp_data.drop([col, 'STDDEV20'], axis=1)

    # Plot SMA20, Upper, Lower

    tp_data.plot()

    # Labels columns and relevant axes
    plt.rcParams["figure.figsize"] = (15,10)
    plt.title('Bollinger ' + col + ' vs. Datetime')
    plt.ylabel(df_list[3], fontsize = 14)
    plt.grid(True)    
    plt.xlim([df_list[1], df_list[2]])
    plt.tight_layout()

    # Save plots
    fig_name = 'BOL_' + col + '_' + df_list[3] + df_list[1] + df_list[2] + '.png'
    
    plt.savefig(fig_name)

    if os.path.isfile(fig_name):
        print('\n'+ fig_name +'. is saved on'+ os.getcwd())
    else:
        raise Exception('can not save file\n')

    # Returns the plot as an object
    return plt.gcf()



Drawing Figures on the GUI Canvas

In [11]:
def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side = 'top', fill = 'both', expand = 1)
    return figure_canvas_agg

In [12]:
# Deletes the figure 
def delete_figure_agg(figure_agg):
    figure_agg.get_tk_widget().forget()
    plt.close('all')

Create a Window Using Layout and Draw Figure

Layout

###### Creates column for the GUI
###### Inspired by: https://youtu.be/XpKtgNasiBw

In [13]:
button_menu_def = [
    ['AAPL'],
    ['ABBV'],
    ['AMZN'],
    ['BAC'], 
    ['BMY'],
    ['BTH-USD'],
    ['BWA'],
    ['ETH-USD'],
    ['GOOG'],
    ['GS'],
    ['HSBC'],
    ['JNJ'],
    ['JPM'],
    ['META'],
    ['MRK'],
    ['MSFT'],
    ['PFE'],
    ['XRP-USD']
]

inputs_column = [    
    [sg.Text('Choose Column Type')],
    [sg.Listbox(values = [['Close'], ['Typical'], ['closeOffHigh'], ['Volatility'], ['LogReturn']], select_mode= 'LISTBOX_SELECT_MODE_SINGLE', size=(30, 6), key = '-COL_LIST-')],
    # [sg.In(size = (25,1), enable_events = True, key = "-COL-")],
    [sg.Text('Choose Graph Type')],
    [sg.Listbox(values = [['Line'], ['Box'], ['PDF'], ['Bollinger']], select_mode= 'LISTBOX_SELECT_MODE_SINGLE', size=(30, 6), key = '-G_LIST-')],
    [sg.Text('Choose Interval')],
    [sg.Radio('1min',"RADIO1",default=True,key='-IN1-'), sg.Radio('5min', "RADIO1",default =False, key='-IN2-')],
    # [
    #     sg.Button(button_text = 'Line Plot', enable_events = True, key = '-LINE-'),
    #     sg.Button(button_text = 'Box Plot', enable_events = True, key = '-BOX-'),
    #     sg.Button(button_text = 'PDF', enable_events = True, key = '-PDF-'),
    # ],
    
    [sg.Text('Choose Stock/Crypto Symbol')],
    [sg.Text('EX: AAPL, 2022-07-07, 2022-07-10')],
    [sg.Text('Date must be within 2 years over 7 day time-period')],

    [sg.Listbox(values = button_menu_def, size=(30, 6), key = '-STOCK_LIST-', select_mode = 'LISTBOX_SELECT_MODE_SINGLE')],

    #[sg.In(size = (25,1), enable_events = True, key = "-STOCK-")],
    [sg.Text('Choose 1st Date')],
    [sg.Input(key='-START-', size=(20,1)), sg.CalendarButton("START DATE", close_when_date_chosen=True, format='%Y-%m-%d', target='-START-', location=(0,0), no_titlebar=False )],
    [sg.Text('Choose 2nd Date')],
    [sg.Input(key='-END-', size=(20,1)), sg.CalendarButton("END DATE", close_when_date_chosen=True, format='%Y-%m-%d', target='-END-', location=(0,0), no_titlebar=False )],
    [sg.Button(button_text = 'Run', key = "-RUN-")]

    
]

canvas_column = [
    [sg.Canvas(size = (1000,1000), key = "-CANVAS-")],
    [sg.Exit()]
]

sg.change_look_and_feel('DarkBlue13')
layout = [
    [
        sg.Column(inputs_column, justification = 'left'),
        sg.VSeparator(),
        sg.Column(canvas_column)
    ]
]

window = sg.Window("PySimpleGUI + MatPlotLib Line Plot", layout, finalize = True, grab_anywhere = True)

fig = None

while True:
    stock_name = ''
    date1 = ''
    date2 = ''
    column = ''
    ints = ''
    function = ''
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    # If the stock input was filled
    # if event == "-LINE-":
    #    function = create_line_plot
    # if event == "-BOX-":
    #     function = create_box_plot
    # if event == "-PDF-":
    #     function = create_pdf
    if values["-IN1-"] == True:
        ints = '1m'
    elif values["-IN2-"] == True:
        ints = '5m'
    if event == "-RUN-":
        # print(values["-STOCK_LIST-"][0][0])
        # print(values["-COL_LIST-"][0][0])
        stock_name = values["-STOCK_LIST-"][0][0]
        date1 = values["-START-"]
        date2 = values["-END-"]
        column = values["-COL_LIST-"][0][0]
        if values["-G_LIST-"][0][0] == "Line":
            function = create_line_plot
        elif values["-G_LIST-"][0][0] == "Box":
            function = create_box_plot
        elif values["-G_LIST-"][0][0] == "PDF":
            function = create_pdf
        elif values["-G_LIST-"][0][0] == "Bollinger":
            function = create_bollinger_bands
        if fig:
            delete_figure_agg(fig)
            window.refresh()
 
    if stock_name != '' and date1 != '' and date2 != '' and function != '' and ints != '':
        plot = function(graph_data(stock_name, ints, date1, date2), column)
        fig = draw_figure(window['-CANVAS-'].TKCanvas, plot)
        window.refresh()
            
window.close()